In [1]:
import json
import os
from pytube import YouTube
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
import pandas as pd
from pytube.exceptions import VideoUnavailable

#### Download videos from CSV

In [2]:
csv_path = 'csvs/15-11-2021-21_40_02_test.csv'
df = pd.read_csv(csv_path)
df = df.reset_index()  # make sure indexes pair with number of rows
df.head()

,index,label,youtube_id,time_start,time_end,split
0,0,ps,4pRPc04xoEo6,87,93,test
1,1,ru,UBNRj1eRWLs5,77,78,test
2,2,ar,UBNRj1eRWLs12,269,285,test
3,3,br,8oJMGgaww704,6,20,test
4,4,cr,qnpMkLKsLCA5,10,12,test


In [3]:
broken_videos = set()

def retry_download(tries, url, file_name, label):
    for i in range(tries):
            try:
                video = YouTube(url)
                yt_video = video.streams.get_highest_resolution()
                yt_video.download(output_path=f'videos/{label}/', filename=f"{file_name}.mp4")

                # download videos into correct folder
                ffmpeg_extract_subclip(filename=f'videos/{label}/{file_name}.mp4', t1=start, t2=end, targetname=f'videos/{label}/{file_name}_{start}_{end}.mp4') 
                os.remove(f'videos/{label}/{file_name}.mp4')
                
            except VideoUnavailable:
                if i < tries:
                    broken_videos.add(file_name)
                    continue

In [4]:
# Iterates all videos in csv
for index, row in df.iterrows():
    file_name = str(row['youtube_id'])
    start = int(row['time_start'])
    end = int(row['time_end'])
    label = str(row['label'])
    url = f'https://www.youtube.com/watch?v={file_name[0:11]}'
    fill_start = str(start).zfill(6)
    fill_end = str(end).zfill(6)

    if os.path.exists(f"videos/{label}/{file_name}_{start}_{end}.mp4"):
            print('file already exists')
            continue
    
    try:
        video = YouTube(url)
        yt_video = video.streams.get_highest_resolution()
        yt_video.download(output_path=f'videos/{label}/', filename=f"{file_name}.mp4")

        # download videos into correct folder
        
        ffmpeg_extract_subclip(filename=f'videos/{label}/{file_name}.mp4', t1=start, t2=end, 
                                targetname=f'videos/{label}/{file_name}_{fill_start}_{fill_end}.mp4') 
        os.remove(f'videos/{label}/{file_name}.mp4')

    except:
        print('retrying...', file_name)
        retry_download(1, url, file_name, label)      

Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
retrying... qnpMkLKsLCA5
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy -

In [ ]:
for video in broken_videos:
    df = df.drop(df.loc[df['youtube_id'] == video].index)

In [23]:
# create label_map.txt
for i in range(2):
    label_map = open('videos/label_map10_.txt', 'a+')
    for i in df['label'].unique():
        label_map.write(f'{i}\n')